In [ ]:
import os
import pandas as pd
import datetime
import glob
from google.colab import files
import numpy as np
import re
#import dask.dataframe as dd
#from dask.diagnostics import ProgressBar
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# LOAD AND PREPROCESS DATA

In [ ]:
i2m = list(zip(range(1,13), ['Gener','Febrer','Marc','Abril','Maig','Juny','Juliol','Agost','Setembre','Octubre','Novembre','Desembre']))
for year in range(2023, 2019, -1):
    for month, month_name in i2m:
        if (month > 5) and (year>2023): continue
        #Descarrega arxius de la web de Bicing
        os.system(f'curl -L -o "{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z" "https://opendata-ajuntament.barcelona.cat/resources/bcn/BicingBCN/{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z"')
        #Extreu fitxers del format 7z
        os.system(f"7z x '{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z'")
        #Elimina l'arxiu comprimit
        os.system(f"rm '{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z'")


In [ ]:
import os
import re
fileList = os.listdir()

# Regex que coincideix amb: 2020_02_Febrer_BicingNou_ESTACIONS.csv
pattern = re.compile(r"^(202[0-3])_\d{2}_[A-Za-zÀ-ÿ]+_BicingNou_ESTACIONS\.csv$")

listCsv = [file for file in fileList if file.endswith(".csv") and pattern.match(file)]
listCsv = sorted(listCsv)
listCsv

['2020_01_Gener_BicingNou_ESTACIONS.csv',
 '2020_02_Febrer_BicingNou_ESTACIONS.csv',
 '2020_03_Marc_BicingNou_ESTACIONS.csv',
 '2020_04_Abril_BicingNou_ESTACIONS.csv',
 '2020_05_Maig_BicingNou_ESTACIONS.csv',
 '2020_06_Juny_BicingNou_ESTACIONS.csv',
 '2020_07_Juliol_BicingNou_ESTACIONS.csv',
 '2020_08_Agost_BicingNou_ESTACIONS.csv',
 '2020_09_Setembre_BicingNou_ESTACIONS.csv',
 '2020_10_Octubre_BicingNou_ESTACIONS.csv',
 '2020_11_Novembre_BicingNou_ESTACIONS.csv',
 '2020_12_Desembre_BicingNou_ESTACIONS.csv',
 '2021_01_Gener_BicingNou_ESTACIONS.csv',
 '2021_02_Febrer_BicingNou_ESTACIONS.csv',
 '2021_03_Març_BicingNou_ESTACIONS.csv',
 '2021_04_Abril_BicingNou_ESTACIONS.csv',
 '2021_05_Maig_BicingNou_ESTACIONS.csv',
 '2021_06_Juny_BicingNou_ESTACIONS.csv',
 '2021_07_Juliol_BicingNou_ESTACIONS.csv',
 '2021_08_Agost_BicingNou_ESTACIONS.csv',
 '2021_09_Setembre_BicingNou_ESTACIONS.csv',
 '2021_10_Octubre_BicingNou_ESTACIONS.csv',
 '2021_11_Novembre_BicingNou_ESTACIONS.csv',
 '2021_12_Desembr

In [ ]:
!pip install dask

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 17.1 MB/s eta 0:00:00
  Attempting uninstall: importlib_metadata
    Found existing installation: importlib-metadata 4.6.4
    Uninstalling importlib-metadata-4.6.4:
      Successfully uninstalled importlib-metadata-4.6.4


In [ ]:
import dask.dataframe as dd
import os
from dask.diagnostics import ProgressBar

def transform_bike_data_dask(listCsv):
    dfs = []
    for file in listCsv:
        print(f"Loading {file}...")
        ddf = dd.read_csv(
            file,
            dtype={
                "station_id": str,
                "last_reported": "float64",
                "is_returning": "object",
                "status": "object"  # ← Añadido para evitar error

            },
            assume_missing=True
        )
        ddf["source_file"] = file  # opcional, para trazabilidad
        dfs.append(ddf)

    # Unir todos los CSVs en un único Dask DataFrame
    ddf = dd.concat(dfs, axis=0)

    # Procesamiento
    ddf["last_updated"] = dd.to_datetime(ddf["last_updated"], unit="s")
    # Usar 'h' en lugar de 'H' para evitar la advertencia futura
    ddf["last_updated_hour"] = ddf["last_updated"].dt.floor("h")

    ddf["total_docks"] = ddf["num_bikes_available"] + ddf["num_docks_available"]
    ddf["percentage_docks_available"] = ddf["num_docks_available"] / ddf["total_docks"]

    # Agrupar por estación y hora
    grouped = (
        ddf.groupby(["station_id", "last_updated_hour"])
        .mean(numeric_only=True)
        .reset_index()
    )

    # Extraer componentes temporales
    grouped["year"] = grouped["last_updated_hour"].dt.year
    grouped["month"] = grouped["last_updated_hour"].dt.month
    grouped["day"] = grouped["last_updated_hour"].dt.day
    grouped["hour"] = grouped["last_updated_hour"].dt.hour

    # Forzar el cómputo para trabajar con Pandas y evitar problemas de índice
    with ProgressBar():
        grouped_pd = grouped.compute()

    # Ordenar por station_id y last_updated_hour para aplicar shift correctamente
    grouped_pd = grouped_pd.sort_values(["station_id", "last_updated_hour"]).reset_index(drop=True)

    # Calcular ctx-0 (valor actual)
    grouped_pd["ctx-0"] = grouped_pd["percentage_docks_available"]

    # Calcular ctx-1 a ctx-4 de forma continua por station_id
    for i in range(1, 5):
        grouped_pd[f"ctx-{i}"] = (
            grouped_pd.groupby("station_id")["percentage_docks_available"]
            .shift(i)
            .fillna(0)
        )

    # Seleccionar columnas finales
    # Seleccionar columnas finales
    final_cols = [
        "station_id", "year", "month", "day", "hour",
        "ctx-4", "ctx-3", "ctx-2", "ctx-1", "ctx-0"
    ]

    final_df = grouped_pd[final_cols]

    return final_df

1. Carregar tots els fitxers CSV en dataframes i, a continuació, calcular la mitjana per hora de les variables (fem la mitjana per a cada estació station_id i hora), intentem usar Dask però no és l'òptim per a nosaltres. A més afegim la columna de mes (month_num) i any (year) per quan unim tots els diferents mesos saber d'on provenien les observacions, i les variables 'ctx-4', 'ctx-3', 'ctx-2', 'ctx-1', 'percentage_docks_available', 'percentage_bikes_mech', 'percentage_bikes_e'.

In [ ]:
listCsv = sorted(listCsv)

df_result = transform_bike_data_dask(listCsv)

Loading 2020_01_Gener_BicingNou_ESTACIONS.csv...
Loading 2020_02_Febrer_BicingNou_ESTACIONS.csv...
Loading 2020_03_Marc_BicingNou_ESTACIONS.csv...
Loading 2020_04_Abril_BicingNou_ESTACIONS.csv...
Loading 2020_05_Maig_BicingNou_ESTACIONS.csv...
Loading 2020_06_Juny_BicingNou_ESTACIONS.csv...
Loading 2020_07_Juliol_BicingNou_ESTACIONS.csv...
Loading 2020_08_Agost_BicingNou_ESTACIONS.csv...
Loading 2020_09_Setembre_BicingNou_ESTACIONS.csv...
Loading 2020_10_Octubre_BicingNou_ESTACIONS.csv...
Loading 2020_11_Novembre_BicingNou_ESTACIONS.csv...
Loading 2020_12_Desembre_BicingNou_ESTACIONS.csv...
Loading 2021_01_Gener_BicingNou_ESTACIONS.csv...
Loading 2021_02_Febrer_BicingNou_ESTACIONS.csv...
Loading 2021_03_Març_BicingNou_ESTACIONS.csv...
Loading 2021_04_Abril_BicingNou_ESTACIONS.csv...
Loading 2021_05_Maig_BicingNou_ESTACIONS.csv...
Loading 2021_06_Juny_BicingNou_ESTACIONS.csv...
Loading 2021_07_Juliol_BicingNou_ESTACIONS.csv...
Loading 2021_08_Agost_BicingNou_ESTACIONS.csv...
Loading 202

/usr/local/lib/python3.11/dist-packages/dask/dataframe/io/csv.py:77: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


[#############################           ] | 73% Completed | 167.33 s

/usr/local/lib/python3.11/dist-packages/dask/dataframe/io/csv.py:77: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


[########################################] | 100% Completed | 196.84 s


In [ ]:
df_result.to_csv("df_result_with_year_2020_2023_v2.csv", index=False)


In [ ]:
df_result

,station_id,year,month,day,hour,ctx-4,ctx-3,ctx-2,ctx-1,ctx-0
0,1,2019.0,12.0,31.0,23.0,0.000000,0.000000,0.000000,0.000000,0.509470
1,1,2020.0,1.0,1.0,0.0,0.000000,0.000000,0.000000,0.509470,0.469697
2,1,2020.0,1.0,1.0,1.0,0.000000,0.000000,0.509470,0.469697,0.407343
3,1,2020.0,1.0,1.0,2.0,0.000000,0.509470,0.469697,0.407343,0.359504
4,1,2020.0,1.0,1.0,3.0,0.509470,0.469697,0.407343,0.359504,0.291667
...,...,...,...,...,...,...,...,...,...,...
16799500,99,2023.0,12.0,31.0,20.0,0.144737,0.208333,0.150000,0.041667,0.016667
16799501,99,2023.0,12.0,31.0,21.0,0.208333,0.150000,0.041667,0.016667,0.045833
16799502,99,2023.0,12.0,31.0,22.0,0.150000,0.041667,0.016667,0.045833,0.000000
16799503,99,2023.0,12.0,31.0,23.0,0.041667,0.016667,0.045833,0.000000,0.000000


In [ ]:
df_result = df_result.dropna()


In [ ]:
df_result

,station_id,year,month,day,hour,ctx-4,ctx-3,ctx-2,ctx-1,ctx-0
0,1,2019.0,12.0,31.0,23.0,0.000000,0.000000,0.000000,0.000000,0.509470
1,1,2020.0,1.0,1.0,0.0,0.000000,0.000000,0.000000,0.509470,0.469697
2,1,2020.0,1.0,1.0,1.0,0.000000,0.000000,0.509470,0.469697,0.407343
3,1,2020.0,1.0,1.0,2.0,0.000000,0.509470,0.469697,0.407343,0.359504
4,1,2020.0,1.0,1.0,3.0,0.509470,0.469697,0.407343,0.359504,0.291667
...,...,...,...,...,...,...,...,...,...,...
16799499,99,2023.0,12.0,31.0,19.0,0.320175,0.144737,0.208333,0.150000,0.041667
16799500,99,2023.0,12.0,31.0,20.0,0.144737,0.208333,0.150000,0.041667,0.016667
16799501,99,2023.0,12.0,31.0,21.0,0.208333,0.150000,0.041667,0.016667,0.045833
16799502,99,2023.0,12.0,31.0,22.0,0.150000,0.041667,0.016667,0.045833,0.000000


# LOAD BIG DATASET, NAS AND CREATE X,Y FOR TRAIN

In [ ]:
#https://www.kaggle.com/competitions/2025-bike-availability-prediction/data?select=metadata_sample_submission_2025.csv

In [ ]:
import pandas as pd

df = pd.read_csv('df_result_with_year_2020_2023_v2.csv')

print(df)

          station_id    year  month   day  hour     ctx-4     ctx-3     ctx-2  \
0                  1  2019.0   12.0  31.0  23.0  0.000000  0.000000  0.000000   
1                  1  2020.0    1.0   1.0   0.0  0.000000  0.000000  0.000000   
2                  1  2020.0    1.0   1.0   1.0  0.000000  0.000000  0.509470   
3                  1  2020.0    1.0   1.0   2.0  0.000000  0.509470  0.469697   
4                  1  2020.0    1.0   1.0   3.0  0.509470  0.469697  0.407343   
...              ...     ...    ...   ...   ...       ...       ...       ...   
16789377          99  2023.0   12.0  31.0  19.0  0.320175  0.144737  0.208333   
16789378          99  2023.0   12.0  31.0  20.0  0.144737  0.208333  0.150000   
16789379          99  2023.0   12.0  31.0  21.0  0.208333  0.150000  0.041667   
16789380          99  2023.0   12.0  31.0  22.0  0.150000  0.041667  0.016667   
16789381          99  2023.0   12.0  31.0  23.0  0.041667  0.016667  0.045833   

             ctx-1     ctx-

In [ ]:
# Contar valores NaN por columna
print("Valores NaN por columna:")
print(df.isnull().sum())

# Contar valores 0 por columna (ten en cuenta que para columnas numéricas)
print("\nValores 0 por columna:")
print((df == 0).sum())

# Eliminar filas con NaNs
df = df.dropna()

# Verificar que se han eliminado los NaNs
print("\nValores NaN por columna después de limpiar:")
print(df.isnull().sum())



Valores NaN por columna:
station_id    0
year          0
month         0
day           0
hour          0
ctx-4         0
ctx-3         0
ctx-2         0
ctx-1         0
ctx-0         0
dtype: int64

Valores 0 por columna:
station_id         0
year               0
month              0
day                0
hour          693944
ctx-4          94750
ctx-3          93904
ctx-2          93032
ctx-1          92127
ctx-0          91123
dtype: int64

Valores NaN por columna después de limpiar:
station_id    0
year          0
month         0
day           0
hour          0
ctx-4         0
ctx-3         0
ctx-2         0
ctx-1         0
ctx-0         0
dtype: int64


In [ ]:
df=df[df['year']>=2021]

In [ ]:
X_train = df
X_train.columns

Index(['station_id', 'year', 'month', 'day', 'hour', 'ctx-4', 'ctx-3', 'ctx-2',
       'ctx-1', 'ctx-0'],
      dtype='object')

# LOAD X TEST

In [ ]:
import pandas as pd

# Ruta al archivo CSV
ruta_csv = "metadata_sample_submission_2025.csv"

# Carga del CSV en un DataFrame
X_test = pd.read_csv(ruta_csv)

# Visualizar las primeras filas
print(X_test.head())

   index  station_id  month  day  hour     ctx-4     ctx-3     ctx-2     ctx-1
0      0           1      6    1     3  0.490942  0.378623  0.324275  0.311594
1      1           1      6    1     8  0.271739  0.311594  0.346014  0.394928
2      2           1      6    1    13  0.538043  0.650362  0.697464  0.721014
3      3           1      6    1    18  0.789855  0.800725  0.791667  0.807971
4      4           1      6    1    23  0.860507  0.871377  0.817029  0.793478


In [ ]:
X_test['year'] = 2024


In [ ]:
X_test

,index,station_id,month,day,hour,ctx-4,ctx-3,ctx-2,ctx-1,year
0,0,1,6,1,3,0.490942,0.378623,0.324275,0.311594,2024
1,1,1,6,1,8,0.271739,0.311594,0.346014,0.394928,2024
2,2,1,6,1,13,0.538043,0.650362,0.697464,0.721014,2024
3,3,1,6,1,18,0.789855,0.800725,0.791667,0.807971,2024
4,4,1,6,1,23,0.860507,0.871377,0.817029,0.793478,2024
...,...,...,...,...,...,...,...,...,...,...
401506,401506,496,12,31,2,0.865741,0.643519,0.597222,0.921296,2024
401507,401507,496,12,31,7,1.000000,0.388889,0.375000,0.407407,2024
401508,401508,496,12,31,12,0.310185,0.337963,0.402778,0.495370,2024
401509,401509,496,12,31,17,0.462963,0.564815,0.629630,0.583333,2024


# WEATHER DATA


## Load data

In [ ]:
import pandas as pd

meteo = pd.read_csv('Informacio_Estacions_Bicing_2025.csv')

meteo = meteo[['station_id', 'lat', 'lon', 'altitude']]



In [ ]:
df_merged = pd.merge(df, meteo, on=['station_id'], how='inner')
df_merged

,station_id,year,month,day,hour,ctx-4,ctx-3,ctx-2,ctx-1,ctx-0,lat,lon,altitude
0,1,2021,1,1,0,0.043478,0.045290,0.050725,0.063406,0.119565,41.397978,2.180107,16.0
1,1,2021,1,1,1,0.045290,0.050725,0.063406,0.119565,0.108696,41.397978,2.180107,16.0
2,1,2021,1,1,2,0.050725,0.063406,0.119565,0.108696,0.108696,41.397978,2.180107,16.0
3,1,2021,1,1,3,0.063406,0.119565,0.108696,0.108696,0.108696,41.397978,2.180107,16.0
4,1,2021,1,1,4,0.119565,0.108696,0.108696,0.108696,0.108696,41.397978,2.180107,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8748579,99,2022,12,31,19,0.468254,0.657738,0.615079,0.686508,0.603175,41.380822,2.141539,30.0
8748580,99,2022,12,31,20,0.657738,0.615079,0.686508,0.603175,0.571429,41.380822,2.141539,30.0
8748581,99,2022,12,31,21,0.615079,0.686508,0.603175,0.571429,0.567460,41.380822,2.141539,30.0
8748582,99,2022,12,31,22,0.686508,0.603175,0.571429,0.567460,0.531746,41.380822,2.141539,30.0


In [ ]:
stations = df_merged[["station_id", "lat", "lon"]].drop_duplicates()
stations

,station_id,lat,lon
0,1,41.397978,2.180107
17392,10,41.346775,2.143623
34827,100,41.379363,2.143868
52240,101,41.392878,2.143411
69663,102,41.392567,2.142217
...,...,...,...
8665623,95,41.376395,2.147327
8683062,96,41.374109,2.148199
8696271,97,41.377961,2.145049
8713705,98,41.380607,2.140863


In [ ]:
from datetime import datetime

# Obtener fecha mínima y máxima del dataset
df_merged["date"] = pd.to_datetime(df_merged[["year", "month", "day"]])

start_date = df_merged["date"].min().strftime("%Y-%m-%d")
end_date = df_merged["date"].max().strftime("%Y-%m-%d")


In [ ]:
import pandas as pd
import os
import requests
from time import sleep

# === Crear carpeta para los CSV meteorológicos ===
os.makedirs("meteo_csv", exist_ok=True)


# === Descargar CSV por estación ===
for _, row in stations.iterrows():
    sid = row["station_id"]
    lat = row["lat"]
    lon = row["lon"]

    output_path = f"meteo_csv/station_{sid}.csv"
    if os.path.exists(output_path):
        print(f"[✓] CSV ya existe para estación {sid}")
        continue

    print(f"[↓] Descargando estación {sid}...")
    url = (
        f"https://archive-api.open-meteo.com/v1/archive?"
        f"latitude={lat}&longitude={lon}"
        f"&start_date={start_date}&end_date={end_date}"
        f"&hourly=temperature_2m,relative_humidity_2m,precipitation"
        f"&timezone=Europe%2FMadrid&format=csv"
    )

    try:
        r = requests.get(url)
        r.raise_for_status()
        with open(output_path, "wb") as f:
            f.write(r.content)
        sleep(1)  # para evitar rate limit
    except Exception as e:
        print(f"Error al descargar estación {sid}: {e}")

[↓] Descargando estación 1.0...
[↓] Descargando estación 10.0...
[↓] Descargando estación 100.0...
[↓] Descargando estación 101.0...
[↓] Descargando estación 102.0...
[↓] Descargando estación 103.0...
[↓] Descargando estación 104.0...
[↓] Descargando estación 105.0...
[↓] Descargando estación 106.0...
[↓] Descargando estación 107.0...
[↓] Descargando estación 108.0...
[↓] Descargando estación 109.0...
[↓] Descargando estación 11.0...
[↓] Descargando estación 110.0...
[↓] Descargando estación 111.0...
[↓] Descargando estación 112.0...
[↓] Descargando estación 113.0...
[↓] Descargando estación 114.0...
[↓] Descargando estación 115.0...
[↓] Descargando estación 116.0...
[↓] Descargando estación 117.0...
[↓] Descargando estación 118.0...
[↓] Descargando estación 119.0...
[↓] Descargando estación 12.0...
[↓] Descargando estación 120.0...
[↓] Descargando estación 121.0...
[↓] Descargando estación 122.0...
[↓] Descargando estación 123.0...
[↓] Descargando estación 124.0...
[↓] Descargando est

In [ ]:
# Eliminar la columna "Unnamed: 0" de X_train
#X_train = X_train.drop(columns=['Unnamed: 0','year'])
#X_test = X_test.drop(columns=['year'])


# Establecer la columna "index" como índice en X_test


In [ ]:
from datetime import datetime

# Asegurar tipo int para las columnas de fecha
X_test["year"] = X_test["year"].astype(int)
X_test["month"] = X_test["month"].astype(int)
X_test["day"] = X_test["day"].astype(int)

# Crear columna de fecha
X_test["date"] = pd.to_datetime(X_test[["year", "month", "day"]])

# Obtener fechas mínima y máxima
start_date_test = X_test["date"].min().strftime("%Y-%m-%d")
end_date_test = X_test["date"].max().strftime("%Y-%m-%d")


In [ ]:
import pandas as pd
import os
import requests
from time import sleep

# === Crear carpeta para los CSV meteorológicos ===
os.makedirs("meteo_test_csv", exist_ok=True)


# === Descargar CSV por estación ===
for _, row in stations.iterrows():
    sid = row["station_id"]
    lat = row["lat"]
    lon = row["lon"]

    output_path = f"meteo_test_csv/station_{sid}.csv"
    if os.path.exists(output_path):
        print(f"[✓] CSV ya existe para estación {sid}")
        continue

    print(f"[↓] Descargando estación {sid}...")
    url = (
        f"https://archive-api.open-meteo.com/v1/archive?"
        f"latitude={lat}&longitude={lon}"
        f"&start_date={start_date_test}&end_date={end_date_test}"
        f"&hourly=temperature_2m,relative_humidity_2m,precipitation"
        f"&timezone=Europe%2FMadrid&format=csv"
    )

    try:
        r = requests.get(url)
        r.raise_for_status()
        with open(output_path, "wb") as f:
            f.write(r.content)
        sleep(1)  # para evitar rate limit
    except Exception as e:
        print(f"Error al descargar estación {sid}: {e}")

In [ ]:
import zipfile
import os

def zip_folder(folder_name, zip_filename):
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(folder_name):
            for file in files:
                filepath = os.path.join(root, file)
                arcname = os.path.relpath(filepath, start=os.path.dirname(folder_name))
                zipf.write(filepath, arcname)

# Comprimir cada carpeta por separado
zip_folder("meteo_csv", "meteo_csv.zip")
zip_folder("meteo_test_csv", "meteo_test_csv.zip")


## Add Columns

In [ ]:
import zipfile

with zipfile.ZipFile("meteo_test_csv.zip", 'r') as zip_ref:
    zip_ref.extractall()

with zipfile.ZipFile("meteo_csv.zip", 'r') as zip_ref:
    zip_ref.extractall()


In [ ]:
import shutil

shutil.rmtree("meteo_test_csv", ignore_errors=True)


In [ ]:
import pandas as pd
from io import StringIO
import os
import re

def transform_csv_to_df(path):
    try:
        with open(path, "r", encoding="utf-8") as f:
            lines = f.readlines()

        # Buscar línea que contenga tanto 'time' como 'temperature'
        header_idx = next(
            i for i, line in enumerate(lines)
            if "time" in line.lower() and "temperature" in line.lower()
        )

        # Extraer cabecera
        header = lines[header_idx].strip().split(",")

        # Datos desde la línea siguiente
        data_lines = lines[header_idx + 1:]

        # Si la primera línea de datos repite la cabecera, saltarla
        first_data_line = data_lines[0].strip().split(",")
        if all(a.strip() == b.strip() for a, b in zip(first_data_line, header)):
            data_lines = data_lines[1:]

        # Leer como CSV
        data_str = "".join(data_lines)
        df = pd.read_csv(StringIO(data_str), header=None)
        df.columns = header[:len(df.columns)]

        # Extraer station_id del nombre del archivo
        filename = os.path.basename(path)
        match = re.search(r"station_(\d+(?:\.\d+)?)", filename)
        station_id = int(float(match.group(1))) if match else None
        df["station_id"] = station_id

        #print("Columnas detectadas:")
        #print(df.columns.tolist())
        #print("\nPrimeras filas:")
        #print(df.head())

        return df

    except Exception as e:
        print(f"[!] Error al leer {path}: {e}")


In [ ]:
df_weather_train = transform_csv_to_df("meteo_csv/station_105.0.csv")
df_weather_train


,time,temperature_2m (°C),relative_humidity_2m (%),precipitation (mm),station_id
0,2021-01-01T00:00,5.7,82,0.0,105
1,2021-01-01T01:00,6.5,79,0.0,105
2,2021-01-01T02:00,5.0,83,0.0,105
3,2021-01-01T03:00,4.7,84,0.0,105
4,2021-01-01T04:00,4.6,86,0.0,105
...,...,...,...,...,...
17515,2022-12-31T19:00,13.7,74,0.0,105
17516,2022-12-31T20:00,12.5,78,0.0,105
17517,2022-12-31T21:00,11.5,85,0.0,105
17518,2022-12-31T22:00,10.5,87,0.0,105


In [ ]:
X_train

,station_id,year,month,day,hour,ctx-4,ctx-3,ctx-2,ctx-1,ctx-0
8163,1,2021,1,1,0,0.043478,0.045290,0.050725,0.063406,0.119565
8164,1,2021,1,1,1,0.045290,0.050725,0.063406,0.119565,0.108696
8165,1,2021,1,1,2,0.050725,0.063406,0.119565,0.108696,0.108696
8166,1,2021,1,1,3,0.063406,0.119565,0.108696,0.108696,0.108696
8167,1,2021,1,1,4,0.119565,0.108696,0.108696,0.108696,0.108696
...,...,...,...,...,...,...,...,...,...,...
12684382,99,2022,12,31,19,0.468254,0.657738,0.615079,0.686508,0.603175
12684383,99,2022,12,31,20,0.657738,0.615079,0.686508,0.603175,0.571429
12684384,99,2022,12,31,21,0.615079,0.686508,0.603175,0.571429,0.567460
12684385,99,2022,12,31,22,0.686508,0.603175,0.571429,0.567460,0.531746


In [ ]:
X_test

,index,station_id,month,day,hour,ctx-4,ctx-3,ctx-2,ctx-1,year
0,0,1,6,1,3,0.490942,0.378623,0.324275,0.311594,2024
1,1,1,6,1,8,0.271739,0.311594,0.346014,0.394928,2024
2,2,1,6,1,13,0.538043,0.650362,0.697464,0.721014,2024
3,3,1,6,1,18,0.789855,0.800725,0.791667,0.807971,2024
4,4,1,6,1,23,0.860507,0.871377,0.817029,0.793478,2024
...,...,...,...,...,...,...,...,...,...,...
401506,401506,496,12,31,2,0.865741,0.643519,0.597222,0.921296,2024
401507,401507,496,12,31,7,1.000000,0.388889,0.375000,0.407407,2024
401508,401508,496,12,31,12,0.310185,0.337963,0.402778,0.495370,2024
401509,401509,496,12,31,17,0.462963,0.564815,0.629630,0.583333,2024


In [ ]:
import pandas as pd
import os
from glob import glob

def merge_meteo_with_X_train(meteo_folder, X_train):
    # 1. Leer todos los CSV de meteo_csv/
    all_csv_paths = glob(os.path.join(meteo_folder, "*.csv"))
    meteo_dfs = [transform_csv_to_df(p) for p in all_csv_paths]

    # 2. Concatenar todos los DataFrames de meteorología
    meteo_df = pd.concat(meteo_dfs, ignore_index=True)

    # 3. Convertir columna "time" en datetime
    meteo_df["time"] = pd.to_datetime(meteo_df["time"])

    # 4. Crear columna datetime en X_train
    X_train = X_train.copy()
    X_train["time"] = pd.to_datetime(X_train[["year", "month", "day", "hour"]])

    # 5. Merge por station_id y time
    merged_df = pd.merge(
        X_train,
        meteo_df[["station_id", "time", "temperature_2m (°C)", "precipitation (mm)"]],
        on=["station_id", "time"],
        how="left"
    )

    # 6. Rellenar NaNs con la media global de cada columna meteorológica
    for col in ["temperature_2m (°C)", "precipitation (mm)"]:
        if col in merged_df.columns:
            merged_df[col].fillna(merged_df[col].mean(), inplace=True)

    # 7. Eliminar columnas no necesarias
    merged_df.drop(columns=["time"], inplace=True)

    return merged_df


In [ ]:
X_train_enriquecido = merge_meteo_with_X_train("meteo_csv", X_train)
X_train_enriquecido

,station_id,year,month,day,hour,ctx-4,ctx-3,ctx-2,ctx-1,ctx-0,temperature_2m (°C),precipitation (mm)
0,1,2021.0,1.0,1.0,0.0,0.043478,0.045290,0.050725,0.063406,0.119565,5.700000,0.000000
1,1,2021.0,1.0,1.0,1.0,0.045290,0.050725,0.063406,0.119565,0.108696,6.500000,0.000000
2,1,2021.0,1.0,1.0,2.0,0.050725,0.063406,0.119565,0.108696,0.108696,5.000000,0.000000
3,1,2021.0,1.0,1.0,3.0,0.063406,0.119565,0.108696,0.108696,0.108696,4.700000,0.000000
4,1,2021.0,1.0,1.0,4.0,0.119565,0.108696,0.108696,0.108696,0.108696,4.600000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
12949695,99,2023.0,12.0,31.0,19.0,0.320175,0.144737,0.208333,0.150000,0.041667,16.683865,0.064191
12949696,99,2023.0,12.0,31.0,20.0,0.144737,0.208333,0.150000,0.041667,0.016667,16.683865,0.064191
12949697,99,2023.0,12.0,31.0,21.0,0.208333,0.150000,0.041667,0.016667,0.045833,16.683865,0.064191
12949698,99,2023.0,12.0,31.0,22.0,0.150000,0.041667,0.016667,0.045833,0.000000,16.683865,0.064191


In [ ]:
X_test_enriquecido = merge_meteo_with_X_train("meteo_test_csv", X_test)
X_test_enriquecido.set_index('index', inplace=True)
X_test_enriquecido

,station_id,month,day,hour,ctx-4,ctx-3,ctx-2,ctx-1,year,temperature_2m (°C),precipitation (mm)
index,,,,,,,,,,,
0,1,6,1,3,0.490942,0.378623,0.324275,0.311594,2024,16.200000,1.40000
1,1,6,1,8,0.271739,0.311594,0.346014,0.394928,2024,18.100000,0.00000
2,1,6,1,13,0.538043,0.650362,0.697464,0.721014,2024,22.200000,0.00000
3,1,6,1,18,0.789855,0.800725,0.791667,0.807971,2024,20.300000,0.80000
4,1,6,1,23,0.860507,0.871377,0.817029,0.793478,2024,16.600000,0.10000
...,...,...,...,...,...,...,...,...,...,...,...
401506,496,12,31,2,0.865741,0.643519,0.597222,0.921296,2024,19.117638,0.10344
401507,496,12,31,7,1.000000,0.388889,0.375000,0.407407,2024,19.117638,0.10344
401508,496,12,31,12,0.310185,0.337963,0.402778,0.495370,2024,19.117638,0.10344


In [ ]:
X_train_end=X_train_enriquecido.drop(columns=['ctx-0'])
X_train_end

,station_id,year,month,day,hour,ctx-4,ctx-3,ctx-2,ctx-1,temperature_2m (°C),precipitation (mm)
0,1,2021.0,1.0,1.0,0.0,0.043478,0.045290,0.050725,0.063406,5.700000,0.000000
1,1,2021.0,1.0,1.0,1.0,0.045290,0.050725,0.063406,0.119565,6.500000,0.000000
2,1,2021.0,1.0,1.0,2.0,0.050725,0.063406,0.119565,0.108696,5.000000,0.000000
3,1,2021.0,1.0,1.0,3.0,0.063406,0.119565,0.108696,0.108696,4.700000,0.000000
4,1,2021.0,1.0,1.0,4.0,0.119565,0.108696,0.108696,0.108696,4.600000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
12949695,99,2023.0,12.0,31.0,19.0,0.320175,0.144737,0.208333,0.150000,16.683865,0.064191
12949696,99,2023.0,12.0,31.0,20.0,0.144737,0.208333,0.150000,0.041667,16.683865,0.064191
12949697,99,2023.0,12.0,31.0,21.0,0.208333,0.150000,0.041667,0.016667,16.683865,0.064191
12949698,99,2023.0,12.0,31.0,22.0,0.150000,0.041667,0.016667,0.045833,16.683865,0.064191


In [ ]:
y_train_end=X_train_enriquecido['ctx-0']
y_train_end

,ctx-0
0,0.119565
1,0.108696
2,0.108696
3,0.108696
4,0.108696
...,...
12949695,0.041667
12949696,0.016667
12949697,0.045833
12949698,0.000000


In [ ]:
X_test_end=X_test_enriquecido

## Save csvs

In [ ]:
X_train_end.to_csv("X_train_end_year_2023.csv", index=False)
y_train_end.to_csv("y_train_end_year_2023.csv", index=False)
#X_test_enriquecido.to_csv("X_test_end_year.csv", index=False)

In [ ]:
X_test_enriquecido.to_csv("X_test_end_year_2023.csv", index=False)

In [ ]:
X_test_enriquecido

,station_id,month,day,hour,ctx-4,ctx-3,ctx-2,ctx-1,year,temperature_2m (°C),precipitation (mm)
index,,,,,,,,,,,
0,1,6,1,3,0.490942,0.378623,0.324275,0.311594,2024,16.200000,1.40000
1,1,6,1,8,0.271739,0.311594,0.346014,0.394928,2024,18.100000,0.00000
2,1,6,1,13,0.538043,0.650362,0.697464,0.721014,2024,22.200000,0.00000
3,1,6,1,18,0.789855,0.800725,0.791667,0.807971,2024,20.300000,0.80000
4,1,6,1,23,0.860507,0.871377,0.817029,0.793478,2024,16.600000,0.10000
...,...,...,...,...,...,...,...,...,...,...,...
401506,496,12,31,2,0.865741,0.643519,0.597222,0.921296,2024,19.117638,0.10344
401507,496,12,31,7,1.000000,0.388889,0.375000,0.407407,2024,19.117638,0.10344
401508,496,12,31,12,0.310185,0.337963,0.402778,0.495370,2024,19.117638,0.10344


# MODEL TESTING

## LOAD CSV

In [ ]:
X_train_end=pd.read_csv("X_train_end_year_2023.csv")
y_train_end=pd.read_csv("y_train_end_year_2023.csv")
X_test_end=pd.read_csv("X_test_end_year.csv")

In [ ]:
X_train_end = X_train_end.drop(columns=['year'])
X_train_end

,station_id,month,day,hour,ctx-4,ctx-3,ctx-2,ctx-1,temperature_2m (°C),precipitation (mm)
0,1,1.0,1.0,0.0,0.043478,0.045290,0.050725,0.063406,5.700000,0.000000
1,1,1.0,1.0,1.0,0.045290,0.050725,0.063406,0.119565,6.500000,0.000000
2,1,1.0,1.0,2.0,0.050725,0.063406,0.119565,0.108696,5.000000,0.000000
3,1,1.0,1.0,3.0,0.063406,0.119565,0.108696,0.108696,4.700000,0.000000
4,1,1.0,1.0,4.0,0.119565,0.108696,0.108696,0.108696,4.600000,0.000000
...,...,...,...,...,...,...,...,...,...,...
12949695,99,12.0,31.0,19.0,0.320175,0.144737,0.208333,0.150000,16.683865,0.064191
12949696,99,12.0,31.0,20.0,0.144737,0.208333,0.150000,0.041667,16.683865,0.064191
12949697,99,12.0,31.0,21.0,0.208333,0.150000,0.041667,0.016667,16.683865,0.064191
12949698,99,12.0,31.0,22.0,0.150000,0.041667,0.016667,0.045833,16.683865,0.064191


In [ ]:
X_test_end=X_test_end.drop(columns=['year'])


## LINEAR REGRESSION

In [ ]:
simple_model = LinearRegression()
simple_model.fit(X_train_end, y_train_end)

LinearRegression()

In [ ]:
y_pred = simple_model.predict(X_test_end)

In [ ]:
y_pred

array([[0.33750107],
       [0.42506177],
       [0.71064824],
       ...,
       [0.53649211],
       [0.58520343],
       [0.55565806]])

In [ ]:
y_pred_end = y_pred.ravel()

# Crear el DataFrame utilizando el índice de X_test
df_output = pd.DataFrame({
    "percentage_docks_available": y_pred_end,
    "index": X_test_end.index
})


df_output


,percentage_docks_available,index
0,0.337501,0
1,0.425062,1
2,0.710648,2
3,0.784458,3
4,0.763257,4
...,...,...
401506,0.965568,401506
401507,0.433248,401507
401508,0.536492,401508
401509,0.585203,401509


In [ ]:
df_output.to_csv("submit_lr_1.csv", index=False)


## RED NEURONAL

In [ ]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import os
from google.colab import files
from sklearn.preprocessing import OneHotEncoder

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

# Escalar y aplicar PCA a X_train
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train_end)

pca = PCA(n_components=0.95)  # Mantener el 95% de la varianza explicada
X_train_scaled = pca.fit_transform(X_train_scaled)

# Transformar X_test con el mismo scaler y PCA
X_test_scaled = scaler.transform(X_test_end)
X_test_pca = pca.transform(X_test_scaled)

# Crear y compilar el modelo de red neuronal
model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1)
])

# Optimizador con decaimiento exponencial del learning rate
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=1000,
    decay_rate=0.9
)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Callback: parar si no mejora tras 3 épocas
early_stop = EarlyStopping(monitor='loss', patience=2, restore_best_weights=True)

# Entrenar el modelo
model.fit(
    X_train_scaled,
    y_train_end,
    epochs=50,
    batch_size=64,
    callbacks=[early_stop],
    verbose=1
)

# Predicción sobre X_test
y_pred = model.predict(X_test_pca)
print(y_pred)


#RANDOM FOREST

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model_RF = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
model_RF.fit(X_train_end, y_train_end)


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor(max_depth=10, random_state=42)

In [ ]:
y_pred = model_RF.predict(X_test_end)

In [ ]:
y_pred_end = y_pred.ravel()

# Crear el DataFrame utilizando el índice de X_test
df_output = pd.DataFrame({
    "percentage_docks_available": y_pred_end,
    "index": X_test_end.index
})


df_output
df_output.to_csv("submit_RF.csv", index=False)


# XGBRegressor

## 1

In [ ]:
!pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.3/201.3 MB 5.0 MB/s eta 0:00:00


In [ ]:
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor

# Estandarización
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_end)
X_test_scaled = scaler.transform(X_test_end)

# Modelo optimizado
model_X = XGBRegressor(
    n_estimators=900,
    learning_rate=0.01,
    max_depth=25,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
model_X.fit(X_train_scaled, y_train_end)


In [ ]:
y_pred = model_X.predict(X_test_scaled
                         )

In [ ]:
import pickle

# Guardar el modelo entrenado
with open("model_X1.pkl", "wb") as f:
    pickle.dump(model_X, f)


In [ ]:
y_pred_end = y_pred.ravel()

df_output = pd.DataFrame({
    "percentage_docks_available": y_pred_end,
    "index": X_test_end.index
})


df_output
df_output.to_csv("submit_4.csv", index=False)


## 2

In [ ]:
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor

# Estandarización
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_end)
X_test_scaled = scaler.transform(X_test_end)

# Entrenamiento del modelo
model_X = XGBRegressor(n_estimators=400, learning_rate=0.05, max_depth=12)
model_X.fit(X_train_scaled, y_train_end)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.05, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=12,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=400,
             n_jobs=None, num_parallel_tree=None, ...)

In [ ]:
y_pred = model_X.predict(X_test_scaled)

In [ ]:
y_pred_end = y_pred.ravel()

# Crear el DataFrame utilizando el índice de X_test
df_output = pd.DataFrame({
    "percentage_docks_available": y_pred_end,
    "index": X_test_end.index
})


df_output
df_output.to_csv("submit_weath_2023-v2.csv", index=False)


In [ ]:
import joblib

# Guardar solo el modelo
joblib.dump(model_X, "modelo_docks.pkl")


['modelo_docks.pkl']

# lightgbm

In [ ]:
!pip install lightgbm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 35.6 MB/s eta 0:00:00


In [ ]:
from lightgbm import LGBMRegressor

model = LGBMRegressor(
    n_estimators=700,
    learning_rate=0.05,
    max_depth=20,
    random_state=42
)
model.fit(X_train_scaled, y_train)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.088647 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1817
[LightGBM] [Info] Number of data points in the train set: 8846081, number of used features: 11
[LightGBM] [Info] Start training from score 0.626299


LGBMRegressor(learning_rate=0.05, max_depth=20, n_estimators=700,
              random_state=42)

In [ ]:
y_pred = model.predict(X_test_scaled)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [ ]:
y_pred_end = y_pred.ravel()

# Crear el DataFrame utilizando el índice de X_test
df_output = pd.DataFrame({
    "percentage_docks_available": y_pred_end,
    "index": X_test.index
})


df_output
df_output.to_csv("submit_v6.csv", index=False)
